In [ ]:
!git clone https://github.com/DrYogurt/Aero-Design-Team-Gamma.git
!pip install poetry==1.4.2

In [ ]:
%cd Aero-Design-Team-Gamma
!poetry config virtualenvs.in-project true
!poetry install --no-ansi

In [ ]:
VENV_PATH = "/content/Aero-Design-Team-Gamma/.venv/lib/python3.11/site-packages"
import os, sys
LOCAL_VENV_PATH = '/content/venv' # local notebook
os.symlink(VENV_PATH, LOCAL_VENV_PATH) # connect to directory in drive
sys.path.insert(0, LOCAL_VENV_PATH)

In [ ]:
from hydrogen_boiloff import FluidProperties, TankGeometry, MaterialProperties,analyze_tank_thermal_performance

In [ ]:
# Example materials (from paper's Appendix A)
materials = {
    "outer_shell": MaterialProperties(
        thermal_conductivity=0.15,  # Carbon epoxy W/m*K
        density=1.6,  # Approximate density
        thickness=0.00127  # m
    ),
    "mylar": MaterialProperties(
        thermal_conductivity=0.14,  # W/m*K
        density=1.4,  # Approximate density
        thickness=25.4e-6  # m
    ),
    "insulation": MaterialProperties(
        thermal_conductivity=0.02594,  # EPS W/m*K
        density=1.6229,  # lbs/ft3
        thickness=0.1  # Initial guess in m
    ),
    "inner_shell": MaterialProperties(
        thermal_conductivity=225.94,  # Aluminum W/m*K
        density=2700,  # kg/m3
        thickness=0.001778  # m
    ),
    "lug": MaterialProperties(
        thermal_conductivity=0.25,  # Nylon W/m*K
        density=1150,  # kg/m3
        thickness=0.0254  # m (1 inch)
    )
}
# Case 1: Cruise conditions, LH2 fuel use, full tank, 50% idle fuel usage
conditions_cruise = {
    "ambient_temp": 218.9,  # K (at 35,000 ft)
    "critical_temp": 28.7,  # K
    "solar_flux": 1367,  # W/m2
    "absorptivity": 0.02,
    "cruise_velocity": 232,  # m/s
    "h_vap": 452.51 - 114.55  # kJ/kg
}

# Air properties at cruise
air_cruise = FluidProperties(
    density=0.38035,  # kg/m3
    dynamic_viscosity=1.39e-5,  # kg*s/m
    heat_capacity=1002,  # J/kg*K
    thermal_conductivity=0.0241  # W/m*K
)

# Tank geometry
tank = TankGeometry(
    diameter=1.939,  # m
    length=16.76  # m
)

# Target boiloff rate (50% of idle fuel usage = 173 lbs/hr)
target_boiloff = 173  # lbs/hr

In [ ]:
# Calculate results
boiloff_rate, thickness = analyze_tank_thermal_performance(
    tank=tank,
    ambient_fluid=air_cruise,
    materials=materials,
    operating_conditions=conditions_cruise,
    target_boiloff_rate=target_boiloff
)

# Print comparison
print("\nCruise Conditions Validation:")
print(f"Paper thickness: 3.97 inches")
print(f"Calculated thickness: {thickness/0.0254:.2f} inches")
print(f"Paper insulation weight: 624 lbs")
#print(f"Tank  Surface Area: {tank.surface_area:.3f} m^2")
weight = tank.surface_area * thickness * materials["insulation"].density * 3.28084**3  # m^3 to ft^3
print(f"Calculated weight: {weight:.0f} lbs")


Cruise Conditions Validation:
Paper thickness: 3.97 inches
Calculated thickness: 3.94 inches
Paper insulation weight: 624 lbs
Calculated weight: 619 lbs
